In [4]:
import pandas as pd
import sqlite3
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from tmdbv3api import TMDb

tmdb = TMDb()
tmdb.api_key = '7761c510d7afbefd6ad4e4cd6bd4d653'

for i in range(len(genres)):
    for j in range(len(genres[i])):
        genres[i][j] = dict[genres[i][j]]

tmdb = pd.read_csv('Data/tmdb.movies.csv', index_col=0)
tmdb.head()

ModuleNotFoundError: No module named 'tmdbsimple'